In [42]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
import numpy as np
from ngsolve import *
from netgen.geom2d import unit_square

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace

# -----------------------------
# 1) High-fidelity model (FOM)
# -----------------------------
mesh = Mesh(unit_square.GenerateMesh(maxh=0.11))
V = H1(mesh, order=2, dirichlet="left|right|bottom|top")
u, v = V.TnT()

# Parameter-independent stiffness (mu = 1)
a0 = BilinearForm(V, symmetric=True)
a0 += grad(u) * grad(v) * dx
a0.Assemble()

# Right-hand side: f
f = LinearForm(V)
f += 32 * (y*(1-y)+x*(1-x)) * v * dx
f.Assemble()

# Full-order solve for a given mu (robust re-assembly)
def solve_full(mu: float) -> np.ndarray:
    a_mu = BilinearForm(V, symmetric=True)
    a_mu += mu * grad(u) * grad(v) * dx
    a_mu.Assemble()
    gfu = GridFunction(V)
    gfu.vec.data = a_mu.mat.Inverse(V.FreeDofs()) * f.vec
    return np.array(gfu.vec)  # 1D numpy array (ndofs,)

# Apply K0 to coefficient vector (uses GridFunction vectors to get proper layout)
def apply_K0(coeffs: np.ndarray) -> np.ndarray:
    gfu_in = GridFunction(V)
    gfu_in.vec[:] = coeffs
    vec_out = gfu_in.vec.CreateVector()
    vec_out.data = a0.mat * gfu_in.vec
    return np.array(vec_out)

# -----------------------------
# 2) Snapshot generation (offline)
# -----------------------------
mus_train = np.linspace(0.1, 10.0, 100)   # ensure multiple distinct parameters
snapshots = [solve_full(mu) for mu in mus_train]
X = np.column_stack(snapshots)           # (ndofs, nsnap)
ndofs, nsnap = X.shape
print(f"ndofs={ndofs}, nsnapshots={nsnap}")

# pyMOR vector space + snapshot array (nsnap vectors in rows)
space = NumpyVectorSpace(ndofs)
snapshots_va = space.from_numpy(X.T)     # shape (nsnap, ndofs)
# -----------------------------
# 3) POD basis construction
# -----------------------------
# Request up to 20 modes, bounded by number of snapshots
req_modes = min(20, len(snapshots_va))
# RB_va, svals = pod(snapshots_va, modes=req_modes)
RB_va, svals = pod(snapshots_va)

# Basis matrix with columns as basis vectors (ndofs, r)
Vrb = RB_va.to_numpy().T                # RB_va.to_numpy(): (r, ndofs)
r = Vrb.shape[1]
print(Vrb.shape)
print(f"Requested modes={req_modes}, obtained modes r={r}")

if r == 0:
    raise RuntimeError("POD returned zero modes (rank-deficient snapshots). "
                       "Add more diverse parameters or check the snapshots.")
if r == 1:
    print("Warning: only 1 POD mode found. Consider adding more/varied parameters.")

# -----------------------------
# 4) Reduced operators (offline)
# -----------------------------
# Compute K0*V once, then Ar0 = V^T (K0 V)
KV = np.column_stack([apply_K0(Vrb[:, j]) for j in range(r)])  # (ndofs, r)
Ar0 = Vrb.T @ KV                                               # (r, r)

# Reduced right-hand side: fr = V^T f
f_vec = np.array(f.vec)             # (ndofs,)
fr = Vrb.T @ f_vec                  # (r,)



ndofs=421, nsnapshots=100


Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

(421, 1)
Requested modes=20, obtained modes r=1


In [51]:
# -----------------------------
# 5) Online solve (ROM)
# -----------------------------
def solve_reduced(mu: float) -> np.ndarray:
    Ar_mu = mu * Ar0
    print(Ar0)
    y = np.linalg.solve(Ar_mu, fr)  # (r,)
    return Vrb @ y                  # lifted full vector (ndofs,)

# Test on an unseen parameter
mu_test = 1.234
u_rom = solve_reduced(mu_test)
u_fom = solve_full(mu_test)
rel_err = np.linalg.norm(u_fom - u_rom)/ np.linalg.norm(u_fom)
print(f"Relative error at mu={mu_test:.3f}: {rel_err:.3e}")

# -----------------------------
# 6) Optional visualization
# -----------------------------
try:
    from ngsolve.webgui import Draw
    gfu_fom = GridFunction(V); gfu_fom.vec[:] = u_fom
    gfu_rom = GridFunction(V); gfu_rom.vec[:] = u_rom
    Draw(gfu_fom, mesh, "FOM")
    Draw(gfu_rom, mesh, "ROM")
except Exception as ex:
    print("Visualization skipped:", ex)

[[0.20013679]]
Relative error at mu=1.234: 4.882e-15


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…